### Intended as a script which generates a .cvs report in one run

In [1]:
import tensorflow as tf;
import numpy as np;
import matplotlib.pyplot as plt;
print(tf.__version__);
import pandas as pd;

from sklearn.preprocessing import StandardScaler;
from sklearn.metrics import mean_squared_error;

2.3.0


In [2]:
pd.set_option('display.max_columns', 300);
pd.set_option('display.max_rows', 300);
tf.keras.backend.set_floatx('float64');

In [3]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

[]

In [4]:
MONTHS = 72;
SPLIT = 60; # 2014-2018: training, 2019: testing.
# BATCH_SIZE = 19; # used in NN_v1
BATCH_SIZE = 30;
WINDOW_SIZE = 3;

TEST_LENGTH = MONTHS - SPLIT;

### Data preparation

In [5]:
multi_data = pd.read_csv('../data/zri_acs_bikeshare_merged.csv', index_col = 0);
zip_ids = multi_data.index.unique();

multi_data.drop(["City", "State", "Metro", "CountyName", "year", "month", "datetime"],\
                 axis = 1, inplace = True);

multi_data.head()

,zri,percent_white,percent_black,percent_asian,percent_hispanic,percent_native_am,percent_other_race,percent_0_17,percent_18_39,percent_40_64,percent_65+,percent_rental_units_vacant,percent_rental_units_occupied,percent_graduate_deg,percent_bachelors,percent_associates,percent_highschool,percent_less_highschool,percent_commute_public_transport,percent_commute_less_30,percent_buildings_less_10_units,percent_buildings_10_19_units,percent_buildings_20_49_units,percent_buildings_50+_units,percent_commute_30_to_59,percent_commute_60_to_89,percent_commute_90_more,percent_new_city,percent_new_unit,percent_units_owner_occupied,median_building_age,income_per_capita,poverty_rate,total_pop,percent_workforce_unemployed,percent_work_from_home,median_age,percent_female,gini_index,percent_not_us_citizen,bs_total_stations,bs_total_systems,has_bike_sharing
zip,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1013,928.0,0.728315,0.020951,0.013732,0.219830,0.0,0.004959,0.230010,0.311464,0.309663,0.148864,0.068131,0.931869,0.028046,0.056892,0.259500,0.384872,0.225671,0.023499,0.741732,0.890613,0.030177,0.034856,0.044355,0.227721,0.017344,0.013031,0.108543,0.040940,0.443651,62.0,20475.25,0.191440,22343.583333,0.104130,0.009346,36.416667,0.509533,0.420742,0.056062,0.0,0.0,0
1013,931.0,0.727052,0.021179,0.013708,0.220867,0.0,0.004917,0.229526,0.311552,0.310406,0.148516,0.067207,0.932793,0.028179,0.057377,0.259450,0.384586,0.225073,0.023365,0.740587,0.889943,0.030154,0.035083,0.044820,0.227757,0.017701,0.013401,0.108828,0.040926,0.441963,62.0,20517.50,0.192359,22296.166667,0.104147,0.009839,36.433333,0.508816,0.420483,0.056165,0.0,0.0,0
1013,934.0,0.725788,0.021408,0.013685,0.221904,0.0,0.004874,0.229041,0.311641,0.311150,0.148168,0.066283,0.933717,0.028313,0.057863,0.259400,0.384301,0.224475,0.023230,0.739441,0.889273,0.030131,0.035311,0.045285,0.227792,0.018058,0.013771,0.109112,0.040912,0.440275,62.0,20559.75,0.193277,22248.750000,0.104165,0.010332,36.450000,0.508100,0.420225,0.056267,0.0,0.0,0
1013,929.0,0.724525,0.021636,0.013661,0.222940,0.0,0.004832,0.228557,0.311729,0.311894,0.147820,0.065359,0.934641,0.028446,0.058348,0.259350,0.384016,0.223877,0.023096,0.738295,0.888603,0.030108,0.035538,0.045750,0.227828,0.018415,0.014142,0.109397,0.040897,0.438587,62.0,20602.00,0.194196,22201.333333,0.104182,0.010825,36.466667,0.507383,0.419967,0.056370,0.0,0.0,0
1013,929.0,0.723261,0.021864,0.013638,0.223977,0.0,0.004789,0.228072,0.311818,0.312638,0.147472,0.064435,0.935565,0.028579,0.058833,0.259299,0.383730,0.223279,0.022961,0.737149,0.887934,0.030086,0.035766,0.046215,0.227864,0.018772,0.014512,0.109682,0.040883,0.436898,62.0,20644.25,0.195114,22153.916667,0.104199,0.011318,36.483333,0.506666,0.419708,0.056472,0.0,0.0,0


In [6]:
FEATURES = multi_data.shape[1] - 1;

In [7]:
# for x, y in dataset_train:
#     print(np.array(x), np.array(y))

### Utility functions

In [8]:
@tf.autograph.experimental.do_not_convert
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series); #(43,)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True);
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1)); #(13,43)
    dataset = dataset.shuffle(shuffle_buffer)\
                     .map(lambda window: (window[:-1, 1:], window[-1][0]));
    dataset = dataset.batch(batch_size).prefetch(1);
    return dataset;

In [9]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time Frame")
    plt.ylabel("ZRI")
    plt.grid(True)

### Neural network center

In [10]:
def NN_model(dataset, termination, test = None):
    class myCallbacks(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            mse = logs.get("mse");
            if(mse < termination):
                print("\nGot an mse at {:.4f} in round {} and stopped training\n".format(mse, epoch));
                self.model.stop_training = True;
            
    tf.keras.backend.clear_session();
    # dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

    callback = myCallbacks();
    
    model = tf.keras.models.Sequential([
#         tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
#                           input_shape=[None]),
#       tf.keras.layers.Conv1D(filters=32, kernel_size=3,
#                           strides=1, padding="causal",
#                           activation="relu",
#                           input_shape=[None, WINDOW_SIZE, FEATURES+1]),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, return_sequences=True, input_shape = [None, WINDOW_SIZE, FEATURES])),
        #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, return_sequences=True)),
        # tf.keras.layers.SimpleRNN(8, return_sequences=True),
        #  tf.keras.layers.SimpleRNN(16, return_sequences=True),
        tf.keras.layers.Dense(8, activation="relu"),
    #   tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1)
        # tf.keras.layers.Lambda(lambda x: x * 2.0)
    ]);
    
    optimizer = tf.keras.optimizers.SGD(learning_rate=3e-4, momentum=0.9)
    model.compile(loss=tf.keras.losses.Huber(),
                  optimizer="adam",
                  metrics=["mae", "mse"])

    model.build((None,WINDOW_SIZE,FEATURES))
    # model.summary()

    if not test: history = model.fit(dataset, epochs=500, callbacks = [callback], verbose = 0);
    else: history = model.fit(dataset, epochs=500, validation_data=test,\
                              callbacks=[callback], verbose = 0);
    return model;

In [11]:
def NN_forecast(model, series_transformed):
    forecast = []
    results = []
    for time in range(MONTHS - WINDOW_SIZE):
        forecast.append(model.predict(series_transformed[np.newaxis, time:time + WINDOW_SIZE, 1:]))

    results = [float(x[-1][0]) for x in forecast];
    actual = list(series_transformed[WINDOW_SIZE:, 0]);
    
    return results, actual; #, pure_forecast;

In [12]:
@tf.autograph.experimental.do_not_convert
def NN_test(ZONE, termination=0, plot=False):
    '''
    Input: ZONE
    Output: the RMSE of a NN model on the predicted train, partially predicted test, and complete predicted test.
    '''
    # Collection of data
    series_frame = multi_data[multi_data.index == ZONE];
    # series_frame.sort_values("datetime", ascending = True, inplace = True);
    
    # Standardization
    scaler = StandardScaler();
    series_transformed = scaler.fit_transform(series_frame);
    
    # Train test split
    series_train = series_transformed[:SPLIT];
    series_test = series_transformed[SPLIT-WINDOW_SIZE:];
    
    # Window the training set to make input of the NN
    dataset_train = windowed_dataset(series_train, WINDOW_SIZE, BATCH_SIZE, 60);
    # dataset_test = windowed_dataset(series_test, WINDOW_SIZE, BATCH_SIZE, 60);
    
    model = NN_model(dataset_train, termination);
    
    time_train = list(range(SPLIT));
    time_test = list(range(SPLIT, MONTHS));
    
    # Forecasting
    # results, actual, pure_forecast = NN_forecast(model, series_transformed);
    results, actual = NN_forecast(model, series_transformed);
    
    # Compute MSE
    MSE_train = mean_squared_error(actual[:-TEST_LENGTH], results[:-TEST_LENGTH])**0.5 * scaler.var_[0]**0.5;
    MSE_pure = mean_squared_error(actual[-TEST_LENGTH:], results[-TEST_LENGTH:])**0.5 * scaler.var_[0]**0.5;
    # MSE_pure = mean_squared_error(actual[-TEST_LENGTH:], pure_forecast[-TEST_LENGTH:])**0.5 * scaler.var_[0]**0.5;
    
    if plot: # If the plot option is selected, plot the graph.
        time_actual = range(WINDOW_SIZE, MONTHS);
        plt.figure(figsize=(10, 6))
        plot_series(time_actual, actual);
        plot_series(time_actual, results);
        # plot_series(time_test, pure_forecast);
        plt.show();

    return MSE_train, MSE_pure,\
        np.array(results[-TEST_LENGTH:]) * scaler.var_[0]**0.5 + scaler.mean_[0];

### The script

In [13]:
score_dict = {"zip":[], "RMSE_train":[], "RMSE_pure":[], "forecast":[]};
with tf.device('/cpu:0'):
    for ZONE in zip_ids:
        %time M_train, M_pure, forecast = NN_test(ZONE, termination=0.0025, plot=False);
        print(ZONE, M_train, M_pure, forecast);

        score_dict["zip"].append(ZONE);
        score_dict["RMSE_train"].append(M_train);
        # score_dict["RMSE_test"].append(M_test);
        score_dict["RMSE_pure"].append(M_pure);
        score_dict["forecast"].append(forecast);


Got an mse at 0.0025 in round 232 and stopped training

Wall time: 14.3 s
1013 3.7024572153400834 43.84518442851086 [1099.64680428 1092.81008999 1087.76661756 1088.51145232 1089.46103001
 1090.67856868 1092.22467894 1094.15527185 1096.51634078 1099.32727002
 1102.53973188 1105.9777376 ]

Got an mse at 0.0025 in round 239 and stopped training

Wall time: 13.7 s
1020 3.8585295808348206 45.64462734426438 [1124.24555148 1122.96459762 1122.38625564 1125.4715992  1128.56776262
 1131.07377075 1132.6956632  1133.41990792 1133.40145033 1132.7980771
 1131.70207881 1130.13923756]

Got an mse at 0.0025 in round 385 and stopped training

Wall time: 17.7 s
1040 2.887930589041274 45.56498139910499 [1092.84379543 1088.6768685  1086.55759327 1074.4868747  1078.65986023
 1083.01779111 1087.53721191 1091.29382038 1094.47675357 1097.5201334
 1100.37671564 1102.79957591]

Got an mse at 0.0025 in round 356 and stopped training

Wall time: 16 s
1085 3.2757126788161184 65.27405083954214 [1142.65670512 1134.0


Got an mse at 0.0025 in round 461 and stopped training

Wall time: 19 s
1852 5.410365393474162 129.9303071168775 [1983.31514198 1977.12085227 1970.64686875 1958.11260031 1945.61299639
 1933.07908167 1920.44950572 1907.73305575 1895.13240949 1882.81924387
 1870.91403485 1860.3938289 ]

Got an mse at 0.0025 in round 440 and stopped training

Wall time: 18.3 s
1854 7.279383663144052 136.66683403551613 [1997.4899861  1991.68340069 1987.44324859 1987.9976329  1987.97470187
 1987.65996178 1987.56630377 1988.76666096 1991.90697863 1998.68868681
 2007.40461292 2013.83974187]

Got an mse at 0.0025 in round 265 and stopped training

Wall time: 13.9 s
1876 6.807299862466596 102.32547275627374 [2181.36660682 2174.12894188 2168.2359834  2166.20653902 2160.60316447
 2147.54787876 2125.03024831 2100.0431719  2076.57144001 2055.61586747
 2037.32556904 2021.81668399]

Got an mse at 0.0025 in round 372 and stopped training

Wall time: 17.6 s
1880 5.888049893281603 90.85000008750556 [2190.7770756  2177.

Wall time: 17.1 s
2131 7.582180549688745 226.74700652587654 [2392.00442914 2393.72363308 2394.5382739  2379.01287711 2377.12840881
 2374.68760711 2372.76616047 2372.14070379 2371.38597977 2370.52383466
 2369.5940686  2368.64321714]

Got an mse at 0.0025 in round 296 and stopped training

Wall time: 14.7 s
2132 8.157770899284031 273.57372119461166 [2358.06873293 2352.41776807 2349.17006199 2353.0478169  2352.8334892
 2352.73449287 2352.73835026 2352.83042246 2352.99750223 2353.22929569
 2353.51837661 2353.85936442]

Got an mse at 0.0025 in round 415 and stopped training

Wall time: 16 s
2134 5.6860261551487525 74.37181696596195 [2544.24515668 2529.08774525 2520.57495211 2520.61299008 2530.52960191
 2550.78153208 2582.03279453 2619.83649007 2648.87262606 2666.08521984
 2674.94904707 2680.19119629]

Got an mse at 0.0025 in round 311 and stopped training

Wall time: 13.7 s
2135 5.372938003690686 168.42217356095315 [2722.37103797 2717.6833572  2709.02138544 2685.82050759 2680.85303196
 2676

Wall time: 19.2 s
2451 3.598483957682218 96.50450391130863 [2428.16808035 2424.65642566 2420.72199329 2419.04795896 2417.52130676
 2416.37156224 2415.74799569 2415.61196431 2415.67730426 2417.28915408
 2421.78987707 2421.72811155]

Got an mse at 0.0025 in round 231 and stopped training

Wall time: 11.5 s
2453 4.7692273194219545 139.4044602286017 [2402.02675336 2393.62421489 2386.05392107 2376.55590396 2370.73390978
 2365.59153529 2361.17299257 2357.41825504 2354.65059419 2352.49033629
 2352.22916977 2351.84824113]

Got an mse at 0.0025 in round 253 and stopped training

Wall time: 13.3 s
2458 4.698120479771647 141.17945825573983 [2543.79935946 2536.4357303  2531.31715215 2525.7985723  2521.13929866
 2514.79492101 2508.1245649  2501.5805122  2495.22290363 2488.55318414
 2483.15052336 2479.66984196]

Got an mse at 0.0025 in round 492 and stopped training

Wall time: 19 s
2460 5.658782410008118 174.6220768080939 [2612.85739539 2602.9338203  2594.66705667 2587.90076602 2585.02186446
 2583.

Wall time: 11 s
2914 3.6198105456910357 65.0101294391003 [1260.35303461 1258.5566495  1256.43686357 1254.31629552 1250.57657125
 1245.03931135 1237.7413988  1228.95797684 1219.26190541 1210.60874689
 1211.05062347 1213.53340364]

Got an mse at 0.0025 in round 316 and stopped training

Wall time: 14.1 s
2920 4.214321466332107 50.795587472107385 [1234.50135867 1234.53726273 1234.39469312 1233.11991734 1231.52921621
 1229.71354131 1227.74201204 1225.66828091 1223.5362068  1221.81476609
 1220.27088895 1218.70767256]

Got an mse at 0.0025 in round 221 and stopped training

Wall time: 10.7 s
3038 5.013411039628779 28.1911165997993 [1351.74146974 1347.7641727  1343.8318582  1351.50055762 1356.59448921
 1360.37841434 1362.98071512 1364.51005335 1364.11792199 1362.81580464
 1360.85136276 1358.41891315]

Got an mse at 0.0025 in round 131 and stopped training

Wall time: 9.09 s
3060 3.94639884030878 57.35403997632076 [1427.90815241 1426.84010296 1425.01587007 1425.43953671 1425.29194269
 1424.584

...


Got an mse at 0.0025 in round 283 and stopped training

Wall time: 9.14 s
92410 8.162181799755638 41.502869526781076 [1298.94152798 1303.80008335 1306.52470987 1303.99854207 1299.86741638
 1294.38717598 1287.73307305 1279.95720894 1271.05367135 1261.13316822
 1250.56451163 1240.06651689]

Got an mse at 0.0024 in round 36 and stopped training

Wall time: 4.87 s
92507 6.077994528657143 149.24382658824746 [1473.06989026 1474.47632087 1475.6818797  1478.33158237 1480.18111627
 1481.1587551  1481.2636008  1480.50679386 1478.88752055 1475.83242173
 1469.62277596 1463.13951662]

Got an mse at 0.0025 in round 258 and stopped training

Wall time: 8.9 s
92602 7.799370508270902 52.55634917689933 [2485.85939643 2495.17935727 2503.79268727 2504.45770914 2502.13775675
 2497.85057977 2492.49918425 2487.6252526  2483.19853726 2478.84495071
 2474.72827222 2470.93202475]

Got an mse at 0.0025 in round 421 and stopped training

Wall time: 11.9 s
92603 5.737364294589015 89.38914743953364 [2873.83109954 2


Got an mse at 0.0025 in round 453 and stopped training

Wall time: 12.5 s
93304 1.8192094094982623 15.211197604292018 [879.49335195 878.3625973  877.64094058 876.47047588 875.013784
 873.21477192 871.04345145 868.52919471 864.75125156 860.36909624
 856.43848247 853.55999906]

Got an mse at 0.0025 in round 435 and stopped training

Wall time: 12.1 s
93305 1.7413494877084394 49.24240176322187 [787.54151003 789.29383414 791.46457329 788.61996723 785.47731365
 782.47447882 779.97051076 778.0875104  776.75506166 775.84322348
 775.2557599  774.81299973]

Got an mse at 0.0025 in round 385 and stopped training

Wall time: 11.4 s
93307 1.8910865696473438 21.845941599236994 [858.29138927 858.75827912 859.09470701 858.14788935 857.1300264
 856.07666736 854.58307076 852.87112657 851.16707199 849.46432139
 847.76121653 846.06828359]
Wall time: 13.4 s
93308 1.5156529771453562 7.532722800867366 [855.89852721 859.55427252 861.46970108 861.27279616 860.13501395
 859.1077146  858.30787998 857.54005126 

Wall time: 13.6 s
94549 90.72153669937813 40.0977555573372 [2761.03065945 2779.76530877 2794.52526096 2787.04262019 2777.37830558
 2765.73383641 2752.30628374 2737.28539797 2720.85842574 2703.21662314
 2684.56099256 2665.10691415]

Got an mse at 0.0025 in round 147 and stopped training

Wall time: 6.86 s
94565 9.482296040192493 38.45058496122046 [2004.40853296 2003.11290213 2003.80526124 2006.15431861 2008.02087068
 2009.47754332 2010.58852237 2011.41326931 2012.00872544 2012.42954723
 2012.72648985 2012.94354756]

Got an mse at 0.0025 in round 467 and stopped training

Wall time: 12.9 s
94568 10.463861047417659 127.75255074064604 [2760.96207689 2761.5777929  2762.13934906 2761.78262349 2760.73507621
 2759.18185481 2757.38821133 2755.60842568 2754.02429453 2752.72921683
 2751.7403631  2751.02561504]

Got an mse at 0.0025 in round 113 and stopped training

Wall time: 6.16 s
94603 13.493983896789889 95.74846332738105 [2357.69913844 2356.12134586 2354.72496386 2357.06654504 2358.70695723



Got an mse at 0.0025 in round 227 and stopped training

Wall time: 8.2 s
96826 5.283580756548292 108.19337229324681 [1530.99909193 1527.4432369  1526.59202997 1536.5760879  1547.68163674
 1557.19250541 1564.85897838 1569.04857647 1572.23802165 1575.12023768
 1577.37669534 1578.93810296]

Got an mse at 0.0025 in round 169 and stopped training

Wall time: 7.38 s
97030 7.246582747437438 73.83093269982668 [1355.5500909  1351.09779615 1348.53186754 1352.38523715 1355.2792632
 1357.33495435 1358.27182727 1358.19049261 1357.57144785 1356.57677036
 1351.0793657  1345.00533949]

Got an mse at 0.0025 in round 140 and stopped training

Wall time: 6.72 s
97035 6.079217379338104 77.29468673554022 [1586.73471242 1579.28654604 1568.41728718 1551.61265365 1537.01793418
 1521.00333605 1515.18901793 1513.74725205 1511.23052427 1507.72699679
 1503.48076403 1498.8248142 ]

Got an mse at 0.0025 in round 360 and stopped training

Wall time: 10.8 s
97062 4.923197029592842 30.261605145179377 [1456.72040962 1


Got an mse at 0.0025 in round 169 and stopped training

Wall time: 7.01 s
98121 11.198470955207759 40.45235558409991 [2314.17169192 2306.27912414 2300.14098156 2306.48522343 2310.69176114
 2313.04311556 2317.55506082 2320.32319327 2320.5243841  2316.74463852
 2312.60695331 2311.15696432]

Got an mse at 0.0025 in round 242 and stopped training

Wall time: 8.74 s
98122 5.264922843835909 114.30308520964331 [1892.36203873 1887.78555017 1886.58215379 1882.24442898 1871.11619993
 1851.80532789 1835.34467028 1820.46428585 1806.6262454  1792.17045708
 1781.45063244 1778.77115656]

Got an mse at 0.0025 in round 224 and stopped training

Wall time: 8.3 s
98133 6.119309479994321 60.480706850296805 [1765.5666176  1765.7620294  1765.65532332 1759.70577105 1752.82165084
 1744.84070961 1734.33665855 1721.08603768 1703.54590988 1681.43068128
 1659.86859222 1643.19516654]

Got an mse at 0.0025 in round 190 and stopped training

Wall time: 7.68 s
98201 5.771954181830905 36.36561152805286 [1485.30495134

In [14]:
print(len(score_dict["RMSE_pure"]),\
      sum(score_dict["RMSE_pure"])/len(score_dict["RMSE_pure"]));

rmse = sum(score_dict["RMSE_pure"])/len(score_dict["RMSE_pure"]);

1301 62.161690988829605


In [15]:
test = pd.DataFrame(score_dict);
test.head()

,zip,RMSE_train,RMSE_pure,forecast
0,1013,3.702457,43.845184,"[1099.6468042831696, 1092.810089988809, 1087.7..."
1,1020,3.858530,45.644627,"[1124.2455514829933, 1122.964597618294, 1122.3..."
2,1040,2.887931,45.564981,"[1092.843795426706, 1088.67686849929, 1086.557..."
3,1085,3.275713,65.274051,"[1142.6567051198563, 1134.0932439486069, 1127...."
4,1104,4.277034,68.174545,"[1108.8211530624837, 1101.3282805491126, 1095...."


In [16]:
test.to_csv('NN_featureonly_window_1_overoverfit_{:.2f}.csv'.format(rmse), index = False)